In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer


file_path = "math_placement.xlsx"

# read the data from the file

df = pd.read_excel(file_path)   


In [23]:

# Remove records without hs_gpa information
df_cleaned = df.dropna(subset=['grade', 'hs_gpa'])

# Define the mapping of grades to grade categories
grade_mapping = {
    "A": "Successful", "B": "Successful", "B+": "Successful",
    "A-": "Successful", "B*": "Successful", "A*": "Successful",
    "B+*": "Successful", "W": "Withdrawn", "C-": "Unsuccessful",
    # ... (include all the mappings here) ...
    "C-*": "Unsuccessful", "I*": "Withdrawn", "NR*": "Withdrawn"
}

# Map the grades to their categories and append as a new column
df_cleaned['grade_category'] = df_cleaned['grade'].map(grade_mapping)

# Display the cleaned and imputed dataframe
df_cleaned.head()


C:\Users\britt\AppData\Local\Temp\ipykernel_37248\3350818361.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['grade_category'] = df_cleaned['grade'].map(grade_mapping)


,Row,pidm,term_code,admit_term,degree.x,major_code,college.x,major.x,stu_type,stu_level.x,...,campus_code,course_number,section_number,subj_code,section_type,credit_levl,course_credits,reg_status,grade,grade_category
7,29,149852,202070,202070.0,Associate of Arts,AA,Gallatin College,Associate of Arts,T,UG,...,ZGC,090,922,M,L,UG,4,RE,B*,Successful
8,30,149852,202070,202070.0,Associate of Arts,AA,Gallatin College,Associate of Arts,T,UG,...,ZGC,063,922,M,L,UG,1,RE,A*,Successful
9,35,210982,202230,202230.0,Bachelor of Science,NURS,College of Nursing,Nursing,T,UG,...,Z,216Q,2,STAT,L,UG,3,RW,B,Successful
10,36,210982,202030,202030.0,Associate of Science,AS,Gallatin College,Associate of Science,N,UG,...,Z,216Q,2,STAT,L,UG,3,RW,B,Successful
24,60,452703,202230,202230.0,Bachelor of Science,PNUR,College of Nursing,Pre-Nursing,N,UG,...,Z,121Q,12,M,L,UG,3,RW,B+,Successful


,Row,pidm,term_code,admit_term,degree.x,major_code,college.x,major.x,stu_type,stu_level.x,...,event,campus_code,course_number,section_number,subj_code,section_type,credit_levl,course_credits,reg_status,grade
10,36,210982,202030,202030.0,Associate of Science,AS,Gallatin College,Associate of Science,N,UG,...,202170EOT,Z,216Q,2,STAT,L,UG,3,RW,B
11,39,324739,202130,202130.0,Associate of Arts,AA,Gallatin College,Associate of Arts,T,UG,...,202270EOT,Z,105Q,2,M,L,UG,3,RW,A
12,41,324739,202370,202370.0,Bachelor of Arts,LSQT,College of Letters & Science,Liberal Studies,T,UG,...,202270EOT,Z,105Q,2,M,L,UG,3,RW,A
13,43,372115,202170,202170.0,Bachelor of Science,FNSC,College of Education/HHD,Food and Nutrition,T,UG,...,202170EOT,Z,161Q,7,M,L,UG,4,RW,A-
14,46,376132,202170,202170.0,Bachelor of Science,PBAC,College of Business,Pre-Business,T,UG,...,202230EOT,ZGC,090,921,M,L,UG,4,RW,B+*
15,47,376132,202170,202170.0,Bachelor of Science,PBAC,College of Business,Pre-Business,T,UG,...,202270EOT,Z,161Q,2,M,L,UG,4,W,W
16,48,376132,202170,202170.0,Bachelor of Science,PBAC,College of Business,Pre-Business,T,UG,...,202330EOT,Z,161Q,1,M,L,UG,4,RW,C-
17,49,376132,202170,202170.0,Bachelor of Science,PBAC,College of Business,Pre-Business,T,UG,...,202230EOT,ZGC,063,921,M,L,UG,2,RW,A*
18,54,400959,202170,202170.0,Bachelor of Science,ESGL,College of Letters & Science,Earth Sciences,T,UG,...,202270EOT,Z,172,5,M,L,UG,4,RW,F
19,55,400959,202170,202170.0,Bachelor of Science,ESGL,College of Letters & Science,Earth Sciences,T,UG,...,202170EOT,Z,151Q,5,M,L,UG,4,RW,A-
